In [ ]:
from __future__ import division
import datetime
import time
import os
import sys

import numpy as np
from scipy import linalg
from matplotlib import rcParams
import matplotlib.pyplot as plt

sys.path.append('../')
from alg_tools_1d import dirac_recon_time, periodicSinc, distance

# various experiment settings
stop_cri = 'max_iter'  # stopping criteria: 1) mse; or 2) max_iter

In [ ]:
def plot_dirac(tk, ak, color='red', marker='*', ax=None, label=''):
    if ax is None:
        fig = plt.figure()
        ax = plt.gca()
    markerline211_1, stemlines211_1, baseline211_1 = \
        ax.stem(tk, ak, label=label)
    plt.setp(stemlines211_1, linewidth=1.5, color=color)
    plt.setp(markerline211_1, marker=marker, linewidth=1.5, markersize=8,
             markerfacecolor=color, mec=color)
    plt.setp(baseline211_1, linewidth=0)
    plt.xlim([0, tau])
    
def plot_diracs(ax1, tk, ak, tk_recon, ak_recon, title):
    c1 = [0, 0.447, 0.741]
    m1 ='^'
    plot_dirac(tk, ak, c1, m1, ax=ax1, label='Original Diracs')
    
    c2 = [0.850, 0.325, 0.098]
    m2 = '*'
    plot_dirac(tk_recon, ak_recon, c2, m2, ax=ax1, label='Estimated Diracs')

    plt.axhline(0, color='k')
    plt.ylim([1.17 * np.min(np.concatenate((ak, ak_recon, np.array(0)[np.newaxis]))),
              1.17 * np.max(np.concatenate((ak, ak_recon, np.array(0)[np.newaxis])))])
    plt.ylabel('amplitudes', fontsize=12)
    ax1.yaxis.set_label_coords(-0.095, 0.5)
    plt.legend(numpoints=1, loc=0, fontsize=9, framealpha=0.3,
               handletextpad=.2, columnspacing=0.6, labelspacing=0.05, ncol=2)
    plt.title(title, fontsize=12)

# Overview 

This notebook explains how to reconstruct a signal consisting of a $\tau$-periodic stream of Diracs at unknown locations. We will apply the following standard FRI-based workflow. 

## 1. Generate signal

We generate the FRI signal which we will then try reconstruct:

<br><center>
$ x = \sum_{k' \in \mathbb{Z}} \sum_{k=1}^{K}  \alpha_k \delta(t - t_k - k' \tau ) $ (1) 
</center>

*CODE: Inspect the signal and make sure you understand its parameters.*

## 2. Simulate measurements

We simulate measurements by constructing a non-uniformly sampled, low-pass filtered signal, and add measurement noise. Mathematically, we measure a discrete signal $y_l$ with elements 

$$ y_l = \sum_{k=1}^{K} \alpha_k \phi(t_l' - t_k), $$ 

where $ \phi $ is the Dirichlet kernel $ \phi(t) = \frac{sin(\pi B t)}{B \tau sin(\pi t / \tau)} $ and $ B $ is the bandwith of the ideal lowpass filter. 
 
## 3. Find standard form of signal

Since the signal it is FRI, we know that we can find a representation of the standard form:

<br><center>
$ \hat{x}_m = \sum_{k=1}^{K} \beta_k  u_k^m $ (2) 
</center>

*PEN AND PAPER: Find values of $\beta_k$ and $u_k$*. 

Since the above holds, we know that the signal can be anihilated by a filter $h$. 

*OPTIONAL: Show that for this simple example, this filter is given by*

$$ H(z) = c_0 \prod_{k=1}^K (1 - e^{-j\frac{2 \pi}{\tau} t_k} z^{-1}) $$

## 4. Find and implement $ G $ 

Since we can only measure y and not Once the signal is in form of 2, we need to identify how it is related to measurements y. 

*PEN AND PAPER: find the expression of matrix X such that $ G \hat{x} = y$*


## 5. Solve optimization

Now we have all the ingredients to solve the optimization of the form: 

<center>
find $ \hat{x}, h $ 
</center>

<center>
such that $ || y - G \hat{x} ||_2 \leq \epsilon $
</center>

<center>
and $ x * h = 0 $
</center>

*CODE: you do not have to implement this part, just inspect the obtained solution and make sure it is correct.*

## 6. Reconstruct original signal

Now that we have extracted the locations of the signal, what would you do to find the signal's amplitudes?  

## 1. 

In [ ]:
np.random.seed(7)

K = 5  # number of Diracs
tau = 1  # period of the Dirac stream

# amplitudes of the Diracs
ak = np.sign(np.random.randn(K)) * (1 + (np.random.rand(K) - 0.5) / 1.)

# locations of the Diracs
tk = np.random.rand(K) * tau

plot_dirac(tk, ak)

## 2.

In [ ]:
np.random.seed(7)

def phi(t):
    ''' Dirichlet kernel evaluated at t.
    '''
    numerator = np.sin(np.pi * B * t)
    denominator = B * tau * np.sin(np.pi * t / tau)
    
    idx = np.abs(denominator) < 1e-12
    
    # TODO WHY? 
    numerator[idx] = np.cos(np.pi * B * t[idx])
    denominator[idx] = np.cos(np.pi * t[idx] / tau)
    
    return numerator / denominator

M = 4 * K # number of Fourier samples (at least K)
# TODO WHY?
B = (2. * M + 1.) / tau  # bandwidth of the sampling filter
# TODO WHY?
L = (2 * M + 1) # number of time samples

# measured signal
t_samp = np.random.rand(L) * tau
tl_grid, tk_grid = np.meshgrid(tk, t_samp)
y_ell_samp = np.inner(ak, phi(tl_grid - tk_grid))

# continuous signal (for plotting only)
t_continuous = np.linspace(0, tau, 1000)
tl_grid, tk_grid = np.meshgrid(tk, t_continuous)
y_ell_continuous = np.inner(ak, phi(tl_grid - tk_grid))

## generate noisy signal
sigma_noise = 1e-2
#sigma_noise = 0

noise = np.random.normal(scale=sigma_noise, loc=0, size=y_ell_samp.shape)
y_ell = y_ell_samp + noise

plt.figure()
plt.plot(t_continuous, y_ell_continuous)
plt.plot(t_samp, y_ell_samp, 'r*', label='samples')
plt.plot(t_samp, y_ell, 'g*', label='noisy samples')
plt.xlabel('t')
plt.legend()

## 3.  

In [ ]:
def get_standard_form(ak, tk):
    ''' 
    :param ak: vector of Dirac amplitudes
    :param tk: vector of Dirac locations

    :return: vector of standard form coefficients
    '''
    ms = np.arange(-np.floor(B * tau / 2.), 1 + np.floor(B * tau / 2.))
    tk_grid, m_grid_gt = np.meshgrid(tk, ms)
    
    x_hat = 1. / tau * np.dot(np.exp(-2j * np.pi / tau * m_grid_gt * tk_grid), ak)
    return x_hat

x_hat = get_standard_form(ak, tk)

## 4. 

In [ ]:
def get_G(t_samp):
    '''
    Compute G such that y=Gx
    
    :param t_samp: 
    
    '''
    m_grid, t_samp_grid = np.meshgrid(np.arange(-np.floor(B * tau / 2.), 1 + np.floor(B * tau / 2.)), t_samp)
    G = 1. / B * np.exp(2j * np.pi / tau * m_grid * t_samp_grid)
    return G

G = get_G(t_samp)

## generate noiseless signal
y_ell_test = np.real(np.dot(G, x_hat))

assert np.isclose(y_ell_samp, y_ell_test).all()

## 5.

In [ ]:
# noise energy, in the noiseless case 1e-10 is considered as 0
noise_level = np.max([1e-10, linalg.norm(noise)])
max_ini = 100  # maximum number of random initialisations
xhat_recon, min_error, c_opt, ini = dirac_recon_time(G, y_ell, K, noise_level, max_ini, stop_cri)

print(r'Noise level: {0:.2e}'.format(noise_level))
print(r'Minimum approximation error |a - Gb|_2: {0:.2e}'.format(min_error))

## 6.

In [ ]:
# reconstruct times from filter

## TODO why? 
z = np.roots(c_opt)
z = z / np.abs(z)
tk_recon = np.real(tau * 1j / (2 * np.pi) * np.log(z))
tk_recon = np.sort(tk_recon - np.floor(tk_recon / tau) * tau)

# location estimation error
t_error = distance(tk_recon, tk)[0]
print('location error:', t_error)

In [ ]:
# reconstruct amplitudes ak

Phi_han = periodicSinc(np.pi * B * (np.reshape(t_samp, (-1, 1), order='F') -
                                      np.reshape(tk_recon, (1, -1), order='F')),
                         B * tau)

tl_grid, tk_grid = np.meshgrid(tk_recon, t_samp)
Phi_recon = phi(tl_grid - tk_grid)

assert np.isclose(Phi_han, Phi_recon).all()

ak_recon = np.real(linalg.lstsq(Phi_recon, y_ell)[0])
a_error = distance(ak_recon, ak)[0]
print('amplitude error:', t_error)

In [ ]:
fig = plt.figure(num=1, figsize=(5.5, 2.5), dpi=90)
ax1 = plt.axes([0.125, 0.59, 0.85, 0.31])

t_error_pow = np.int(np.floor(np.log10(t_error)))
title = 'reconstructed vs. original signal'
plot_diracs(ax1, tk, ak, tk_recon, ak_recon, title)